In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [1]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = (df.lpep_dropoff_datetime - df.lpep_pickup_datetime)
    df.duration=df.duration.apply(lambda td: td.total_seconds()/60)
    
    df=df[(df.duration >=1) & (df.duration<=64)]
    
    categorical_columns=['PULocationID', 'DOLocationID']
    df[categorical_columns] = df[categorical_columns].astype(str)
    
    return df

In [3]:
def preprocessing(df, si=None, dv=None):
    
    #df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    
    categorical_columns=['PULocationID', 'DOLocationID']
    #categorical_columns=['PU_DO']
    numerical_columns=['trip_distance', 'passenger_count']
    
   
    
    target='duration'
    y = df[target].values
        
    if dv==None and si==None:
        si = SimpleImputer(missing_values=np.nan, strategy='mean')
        df.passenger_count = si.fit_transform(df.passenger_count.values.reshape(-1,1))
        
        dicts=df[categorical_columns+numerical_columns].to_dict(orient='records')
        dv = DictVectorizer()
        X = dv.fit_transform(dicts)
        
        return X, y, si, dv
    else:
        df.passenger_count = si.transform(df.passenger_count.values.reshape(-1,1))
        
        dicts=df[categorical_columns+numerical_columns].to_dict(orient='records')
        X = dv.transform(dicts)
        
        return X, y
    

In [4]:
def train_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    error = mean_squared_error(y_test, y_pred, squared=False)
    
    return model, error

# Q1. Downloading the data

In [2]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [3]:
len(df.columns)

19

### Answer: 19

# Q2. Computing duration

In [4]:
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
df.duration=df.duration.apply(lambda td: td.total_seconds()/60)

In [5]:
df['duration'].describe()

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
25%      6.316667e+00
50%      1.018333e+01
75%      1.616667e+01
max      8.513183e+03
Name: duration, dtype: float64

### Answer: 46.45 

# Q3. Dropping outliers

In [6]:
((df.duration >=1) & (df.duration<=60)).mean()

0.9827547930522406

In [7]:
df=df[(df.duration >=1) & (df.duration<=60)]

### Answer: 98%

# Q4. One-hot encoding

In [8]:
categorical_columns=['PULocationID', 'DOLocationID']
df[categorical_columns] = df[categorical_columns].astype(str)

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
dicts=df[categorical_columns].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

In [12]:
X_train.shape

(2421440, 515)

In [13]:
target='duration'
y_train = df[target].values

### Answer: 515

# Q5. Training a model

In [29]:
from sklearn.linear_model import LinearRegression

In [38]:
model = LinearRegression() 
model.fit(X_train, y_train)

LinearRegression()

In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
y_pred = model.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

6.986190769549357

### Answer: 6.99

# Q6. Evaluating the model

In [41]:
df_val = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

In [42]:
df_val['duration'] = (df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime)
df_val.duration=df_val.duration.apply(lambda td: td.total_seconds()/60)
df_val=df_val[(df_val.duration >=1) & (df_val.duration<=60)]
dicts=df_val[categorical_columns].to_dict(orient='records')
X_val = dv.transform(dicts)
y_val = df_val[target].values

In [43]:
y_pred = model.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

14.619069996595982

### Answer: 14.619069996595982